Install Packages


In [1]:
#Uncomment if you need to install any or all

# !pip install benepar
# !pip install -q spacy
# !pip install -q spacy-lookups-data

# !pip install -q transformers
# !pip install huggingface_hub


# # !pip install pydot
# # !pip install evaluate

# !pip install tensorflow
# !pip install scipy

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     -------------------------------------- 134.8/134.8 kB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------  1.5/1.5 MB 31.8 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 24.2 MB/s eta 0:00:00
   ---------------------------------

In [270]:
import spacy, benepar
!python -m spacy download en_core_web_md

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

import torch

from transformers import BertTokenizer, BertModel

from tqdm import tqdm

from scipy.spatial.distance import cosine

     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     ---------------------------------------- 0.3/42.8 MB 8.9 MB/s eta 0:00:05
      --------------------------------------- 0.8/42.8 MB 10.0 MB/s eta 0:00:05
     - -------------------------------------- 1.5/42.8 MB 13.5 MB/s eta 0:00:04
     -- ------------------------------------- 2.4/42.8 MB 17.1 MB/s eta 0:00:03
     --- ------------------------------------ 3.6/42.8 MB 19.3 MB/s eta 0:00:03
     ---- ----------------------------------- 5.2/42.8 MB 23.7 MB/s eta 0:00:02
     ------ --------------------------------- 6.9/42.8 MB 27.5 MB/s eta 0:00:02
     -------- ------------------------------- 8.6/42.8 MB 28.9 MB/s eta 0:00:02
     --------- ----------------------------- 10.0/42.8 MB 30.3 MB/s eta 0:00:02
     ---------- ---------------------------- 11.3/42.8 MB 38.6 MB/s eta 0:00:01
     ----------- --------------------------- 12.8/42.8 MB 40.9 MB/s eta 0:00:01
     ------------- ------------------------- 14.6

In [271]:
# Define the Models
model_path = './bert_model_weights'

spacy_checkpoint = "en_core_web_md"
# benepar.download('benepar_en3')  ###Uncommment to download if it's a first run

nlp = spacy.load(spacy_checkpoint)

if spacy.__version__.startswith('2'):
  nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
else:
  nlp.add_pipe("benepar", config={"model": "benepar_en3"})

tokenizer = BertTokenizer.from_pretrained('./bert_model_weights')
model = BertModel.from_pretrained('./bert_model_weights')

In [40]:
train_df = pd.read_json('ROCStories_winter2017_mixed.json')
train_df

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,ending0,ending1,label,story,choices
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...","In the end, he passed with flying colours.",0,David noticed he had put on a lot of weight re...,"[After a few weeks, he started to feel much be..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...,Frank is glad he exercises more now.,0,Tom had a very short temper. One day a guest m...,[Tom sat on his couch filled with regret about...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...,She threw up and felt better.,0,Marcus needed clothing for a business casual e...,[Marcus was happy to have the right clothes fo...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...,She was then able to print the documents with ...,0,Bobby thought Bill should buy a trailer and ha...,[He ended up buying the truck he wanted despit...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,I drove home disappointed that my night didn't...,His congregation was delighted and so was he.,1,John was a pastor with a very bad memory. He t...,[I drove home disappointed that my night didn'...
...,...,...,...,...,...,...,...,...,...,...,...
52660,134e8636-3617-43d8-ba6a-9a11b3b115b1,Flavor,The man liked the flavor.,He tried to recreate it at home.,He could not get the flavor right.,He asked the owner of the recipe for help.,She thanked me for the recommendation.,The owner of the flavor sold him the recipe.,1,The man liked the flavor. He tried to recreate...,"[She thanked me for the recommendation., The o..."
52661,4c317f76-ca42-4024-a4c2-12ec911cf89b,After Death,"After my friend's dad's funeral, I got in trou...",The principal said I wasn't allowed to leave s...,He found out I had my friend sign me out.,He told me I was getting detention.,He barely made it to his relative's.,I skipped detention all week.,1,"After my friend's dad's funeral, I got in trou...","[He barely made it to his relative's., I skipp..."
52662,a18fd0d2-4d0c-4316-befe-e3d827fe699b,Janice breaks her wrist,Janice was out exercising for her big soccer g...,She was doing some drills with her legs.,While working out and exercising she slips on ...,She falls down and uses her wrist to break her...,The coral and the fish were all so beautiful u...,She breaks her wrist in the process and goes t...,1,Janice was out exercising for her big soccer g...,[The coral and the fish were all so beautiful ...
52663,2c14252b-4080-4fca-8765-537772018508,Jamie marries for love,Jamie is an american girl.,Jamie wants to get married to a mexican man.,Her family assumes it's because the man wants ...,Jamie insist that she is marrying him out of l...,Tim grinned as he saw his surprise party!,Jamie gets married and they spent the rest of ...,1,Jamie is an american girl. Jamie wants to get ...,"[Tim grinned as he saw his surprise party!, Ja

In [ ]:
#Fixing Data. These sentences have weird formatting and need to be fixed before extracting triples.
train_df.at[46382,'sentence4'] = 'Although dejected, Jared kept his head high.'
train_df.at[46403,'sentence1'] = 'My company had a "forty hour per week" work policy.'
train_df.at[46724,'sentence4'] = 'When he placed one sticker, it stuck to his hand.'
train_df.at[49377,'sentence3'] = "When the name was called for the award though, it wasn't his."

In [272]:
# To Help Visualize Dependency Tree
import nltk
from nltk import Tree, ParentedTree

from spacy import displacy

In [273]:
# Example of Parser Working on Data
doc = nlp(train_df.loc[0,'story'])
sent = list(doc.sents)[0]
x = sent._.parse_string
print (x)
parse_tree = ParentedTree.fromstring('(' + sent._.parse_string + ')')
print(parse_tree.pretty_print())
print (sent._.parse_string)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(S (NP (NNP David)) (VP (VBD noticed) (SBAR (S (NP (PRP he)) (VP (VBD had) (VP (VBN put) (PRT (RP on)) (NP (NP (DT a) (NN lot)) (PP (IN of) (NP (NN weight)))) (ADVP (RB recently))))))) (. .))
                                                                          
                            |                                              
                            S                                             
   _________________________|___________________________________________   
  |            VP                                                       | 
  |       _____|_______                                                 |  
  |      |            SBAR                                              | 
  |      |             |                                                |  
  |      |             S                                                | 
  |      |      _______|________                                        |  
  |      |     |                VP                   

c:\Users\gonza\Desktop\Project\.venv\lib\site-packages\torch\distributions\distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


In [43]:
displacy.render(doc, style='dep')

In [274]:
#Code to Create Dependency
def create_dependency(df):
    df['dependency_tree'] = None
    for index, row in df.iterrows():
      doc = nlp(row['story'])
      dependency = [sent._.parse_string for sent in doc.sents]
      df.at[index, 'dependency_tree'] = dependency
    return df

In [321]:
#Check to see if dependency_tree is being made
create_dependency(train_df[0:1])

c:\Users\gonza\Desktop\Project\.venv\lib\site-packages\torch\distributions\distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,ending0,ending1,label,story,choices,dependency_tree
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...","In the end, he passed with flying colours.",0,David noticed he had put on a lot of weight re...,"[After a few weeks, he started to feel much be...",[(S (NP (NNP David)) (VP (VBD noticed) (SBAR (...


In [47]:
# Example data
data = {
    "storytitle": "David Drops the Weight",
    "sentences": [
        "David noticed he had put on a lot of weight recently.",
        "He examined his habits to try and figure out the reason.",
        "He realized he'd been eating too much fast food lately.",
        "He stopped going to burger places and started a vegetarian diet.",
        "After a few weeks, he started to feel much better.",
        "Tom sat on his couch filled with regret about his actions."  # This is the incorrect ending
    ]
}

# Function to extract triples from a sentence using spaCy
def extract_triples(sentence):
    doc = nlp(sentence)
    triples = []
    subject = ""
    predicate = ""
    object_ = ""
    for token in doc:
        if token.dep_ == 'ROOT':
          predicate += token.text
        for child in token.children:
            if child.dep_ == 'nsubj' and len(subject) == 0:
                subject += child.text
                subject += " "
            elif child.dep_ == 'dobj' or child.dep_ == 'pobj':
                object_ += child.text
                object_ += " "
    triples.append((subject.strip(), predicate.strip(), object_.strip()))
    return triples

# Extract triples from sentences
triples = []
for sentence in data["sentences"]:
    triples.extend(extract_triples(sentence))



In [48]:
# Tokenize and encode triples
encoded_triples = []
for triple in triples[0:4]:
    encoded_triple = tokenizer([triple[0], triple[1], triple[2]], is_split_into_words = True, return_tensors='pt', padding=True, truncation=True)
    encoded_triples.append(encoded_triple)

# Pass encoded triples through BERT
with torch.no_grad():
    embeddings = []
    for encoded_triple in encoded_triples:
        input_ids = encoded_triple['input_ids'].to(model.device)
        attention_mask = encoded_triple['attention_mask'].to(model.device)
        token_type_ids = encoded_triple['token_type_ids'].to(model.device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling
        embeddings.append(pooled_output)

# Compute similarities between triples and the incorrect ending
wrong_ending_similar = []
for i in range(len(embeddings)):
    encoded_sentence = tokenizer(data["sentences"][-1:], return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**encoded_sentence)
        pooled_output = torch.mean(outputs.last_hidden_state, dim=1).squeeze()  # Mean pooling and squeeze
    sim = 1 - cosine(embeddings[i].squeeze(), pooled_output)  # Compare with the last sentence (incorrect ending) and squeeze
    wrong_ending_similar.append((triples[i], sim))

# Compute similarities between triples and the correct ending
right_ending_similar = []
for i in range(len(embeddings)):
    encoded_sentence = tokenizer(data["sentences"][-2], return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**encoded_sentence)
        pooled_output = torch.mean(outputs.last_hidden_state, dim=1).squeeze()  # Mean pooling and squeeze
    sim = 1 - cosine(embeddings[i].squeeze(), pooled_output)  # Compare with the 2nd last sentence (correct ending) and squeeze
    right_ending_similar.append((triples[i], sim))


# Rank triples based on similarities
wrong_ranked_triples = sorted(wrong_ending_similar, key=lambda x: x[1], reverse=True)
right_ranked_triples = sorted(right_ending_similar, key=lambda x: x[1], reverse=True)

# Print Wrong Ending ranked triples
print("Wrong Ranked Triples:")
sim = []
for idx, (triple, sim) in enumerate(wrong_ranked_triples, start=1):
    print(f"Rank {idx}: Similarity {sim:.4f} - {triple}")
print ("Average:", np.average(sim))

# Print Right Ending ranked triples
sim = []
print("\nRight Ranked Triples:")
for idx, (triple, sim) in enumerate(right_ranked_triples, start=1):
    print(f"Rank {idx}: Similarity {sim:.4f} - {triple}")
print ("Average:", np.average(sim))

# Print correct and incorrect endings
print("\nCorrect Ending:", data["sentences"][-2])  # Last sentence (correct ending)
print("Incorrect Ending:", data["sentences"][-1])  # Last sentence (incorrect ending)

Wrong Ranked Triples:
Rank 1: Similarity 0.3878 - ('David', 'noticed', 'lot weight')
Rank 2: Similarity 0.3740 - ('He', 'stopped', 'places diet')
Rank 3: Similarity 0.3725 - ('He', 'realized', 'food')
Rank 4: Similarity 0.3676 - ('He', 'examined', 'habits reason')
Average: 0.36759167010826643

Right Ranked Triples:
Rank 1: Similarity 0.3873 - ('He', 'realized', 'food')
Rank 2: Similarity 0.3852 - ('David', 'noticed', 'lot weight')
Rank 3: Similarity 0.3833 - ('He', 'examined', 'habits reason')
Rank 4: Similarity 0.3767 - ('He', 'stopped', 'places diet')
Average: 0.3766734655919215

Correct Ending: After a few weeks, he started to feel much better.
Incorrect Ending: Tom sat on his couch filled with regret about his actions.


In [49]:
#Create Triple for Story in DF
def df_graph(df):
  """"Function used to create triples for the story. It returns the df w/ a new column for triples.
  It's not used further below. Function was used earlier to create a new json file to save time.
  ROCStories_winter2017_mixed.json -> ROCStories_winter2017_mixed_triple.json"""
  
  dic = {'sentences':[]}
  df['story_triple'] = None
  for index, row in tqdm(df.iterrows(),desc = 'Graph',total= len(df)):
    dic['sentences'].append(df.at[index,'sentence1'])
    dic['sentences'].append(df.at[index,'sentence2'])
    dic['sentences'].append(df.at[index,'sentence3'])
    dic['sentences'].append(df.at[index,'sentence4'])

    triples = []

    for sentence in dic['sentences']:
      triples.extend(extract_triples(sentence))
    df.at[index,'story_triple'] = triples
    dic = {'sentences':[]}
  return df

In [50]:
# Use a small version of the dataset to test all functions first.
sample_train = df_graph(train_df[0:5])
sample_train

C:\Users\gonza\AppData\Local\Temp\ipykernel_1360\430651949.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['story_triple'] = None
Graph:   0%|          | 0/5 [00:00<?, ?it/s]c:\Users\gonza\Desktop\Project\.venv\lib\site-packages\torch\distributions\distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(
Graph: 100%|██████████| 5/5 [00:00<00:00,  5.30it/s]


,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,ending0,ending1,label,story,choices,story_triple
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...","In the end, he passed with flying colours.",0,David noticed he had put on a lot of weight re...,"[After a few weeks, he started to feel much be...","[(David, noticed, lot weight), (He, examined, ..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...,Frank is glad he exercises more now.,0,Tom had a very short temper. One day a guest m...,[Tom sat on his couch filled with regret about...,"[(Tom, had, temper), (guest, made, ), (He, pun..."
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...,She threw up and felt better.,0,Marcus needed clothing for a business casual e...,[Marcus was happy to have the right clothes fo...,"[(Marcus, needed, clothing event), (All, were,..."
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...,She was then able to print the documents with ...,0,Bobby thought Bill should buy a trailer and ha...,[He ended up buying the truck he wanted despit...,"[(Bobby, thought, trailer it car), (Bill, thou..."
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,I drove home disappointed that my night didn't...,His congregation was delighted and so was he.,1,John was a pastor with a very bad memory. He t...,[I drove home disappointed that my night didn'...,"[(John, was, memory), (He, tried, sermons adva..."


In [51]:
def tokenize_for_cosine(df):
  """Tokenizes the triples and story endings. Prepares the data for cosine similarity comparison."""

  df['triple_embedding']= None
  df['choices_outputs']= None
  
  for index, row in tqdm(df.iterrows(),desc = 'Tokenizing',total= len(df)):
    encoded_triples = []
    for triple in df.at[index,'story_triple']:
        encoded_triple = tokenizer([triple[0], triple[1], triple[2]], is_split_into_words = True, return_tensors='pt', padding=True, truncation=True)
        encoded_triples.append(encoded_triple)
    # Pass encoded triples through BERT
    with torch.no_grad():
        embeddings = []
        for encoded_triple in encoded_triples:
            input_ids = encoded_triple['input_ids'].to(model.device)
            attention_mask = encoded_triple['attention_mask'].to(model.device)
            token_type_ids = encoded_triple['token_type_ids'].to(model.device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            pooled_output = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling
            embeddings.append(pooled_output)
        df.at[index,'triple_embedding'] = embeddings

    choice_output = []
    for choice in df.at[index,'choices']:
      encoded_sentence = tokenizer(choice, return_tensors='pt', padding=True, truncation=True)  # Change here
      with torch.no_grad():
        outputs = model(**encoded_sentence)
        pooled_output = torch.mean(outputs.last_hidden_state, dim=1).squeeze()
        choice_output.append(pooled_output)
    df.at[index,'choices_outputs'] = choice_output
  return df

In [53]:
pd.options.mode.chained_assignment = None
token_sample_train = tokenize_for_cosine(sample_train)
token_sample_train

Tokenizing: 100%|██████████| 5/5 [00:01<00:00,  4.92it/s]


,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,ending0,ending1,label,story,choices,story_triple,triple_embedding,choices_outputs
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...","In the end, he passed with flying colours.",0,David noticed he had put on a lot of weight re...,"[After a few weeks, he started to feel much be...","[(David, noticed, lot weight), (He, examined, ...","[[[tensor(-0.0005), tensor(0.0173), tensor(0.2...","[[tensor(0.0843), tensor(-0.2282), tensor(0.17..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...,Frank is glad he exercises more now.,0,Tom had a very short temper. One day a guest m...,[Tom sat on his couch filled with regret about...,"[(Tom, had, temper), (guest, made, ), (He, pun...","[[[tensor(-0.0326), tensor(0.1297), tensor(-0....","[[tensor(0.0746), tensor(0.3324), tensor(-0.05..."
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...,She threw up and felt better.,0,Marcus needed clothing for a business casual e...,[Marcus was happy to have the right clothes fo...,"[(Marcus, needed, clothing event), (All, were,...","[[[tensor(0.2004), tensor(0.2414), tensor(-0.1...","[[tensor(0.3946), tensor(-0.3063), tensor(-0.0..."
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...,She was then able to print the documents with ...,0,Bobby thought Bill should buy a trailer and ha...,[He ended up buying the truck he wanted despit...,"[(Bobby, thought, trailer it car), (Bill, thou...","[[[tensor(0.1269), tensor(0.1247), tensor(0.15...","[[tensor(-0.2134), tensor(-0.3301), tensor(0.0..."
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,I drove home disappointed that my night didn't...,His congregation was delighted and so was he.,1,John was a pastor with a very bad memory. He t...,[I drove home disappointed that my night didn'...,"[(John, was, memory), (He, tried, sermons adva...","[[[tensor(0.0861), tensor(-0.1493), tensor(-0....","[[tensor(0.0485), tensor(-0.6491), tensor(-0.1..."


In [55]:
def cosine_scoring(df):
  """Compares the triples to the right and wrong ending."""
  
  df['average'] = None
  for index, row in tqdm(df.iterrows(),desc = 'Scoring',total= len(df)):
    ending0_avg = []
    ending1_avg = []
    for x in df.at[index,'triple_embedding']:
      ending0_avg.append(1 - cosine(x.squeeze(), df.at[index,'choices_outputs'][0]))
      ending1_avg.append(1 - cosine(x.squeeze(), df.at[index,'choices_outputs'][1]))
    ending0_avg = np.average(ending0_avg)
    ending1_avg = np.average(ending1_avg)
    df.at[index,'average'] = [ending0_avg,ending1_avg]
  return df

In [56]:
finish_train = cosine_scoring(token_sample_train[:1])

Scoring: 100%|██████████| 1/1 [00:00<00:00, 1008.25it/s]


In [36]:
def get_accuracy(df):
  """Function to calculate accuracy for the inference model."""
  total = len(df)
  positive = 0
  tp = 0
  fp = 0
  fn = 0
  tn = 0
  for index, row in df.iterrows():
    if np.max(df.at[index,'average']) == df.at[index,'average'][df.at[index,'label']]:
      positive += 1
      tp += 1 
      tn += 1
    else:
      fp += 1
      fn += 1
  negative = total-positive
  accuracy = (tp+tn)/(tp+tn+fn+fp)
  recall = tp/(tp+fn)
  return tp, tn, fp, fn, accuracy, recall

In [58]:
def all_together(df):
  """This function will package together all the functions created to produce a final result.
  The df must have columns: sentence1,	sentence2,	sentence3,	sentence4, choices, and label.
  Not included in here df_graph. In an effort to save time that has been used already and results were stored in json"""
  
  df = tokenize_for_cosine(df)
  print("\nDone w/ Token\n")
  df = cosine_scoring(df)
  print("\nDone w/ Scoring\n")
  final = get_accuracy(df)
  return final

In [61]:
df_triple = pd.read_json('ROCStories_winter2017_mixed_tripled.json')
all_together(df_triple)

Tokenizing: 100%|██████████| 52665/52665 [2:34:17<00:00,  5.69it/s]  



Done w/ Token



Scoring: 100%|██████████| 52665/52665 [00:30<00:00, 1754.71it/s]



Done w/ Scoring



(37729, 37729, 14936, 14936, 0.7163960884838128, 0.7163960884838128)

Seq2Seq Using Triples

In [110]:
# Model Initializing
# !pip install tf-keras

from transformers import TFBertForSequenceClassification

id2label = {0: "Invalid", 1: "Valid"}
label2id = {val: key for key, val in id2label.items()}

bert_seq_model = TFBertForSequenceClassification.from_pretrained(
    './bert_model_weights', num_labels=2, id2label=id2label, label2id=label2id)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Process Datafram to Have Single Ending

In [111]:
df_triple = pd.read_json('ROCStories_winter2017_mixed_tripled.json')
df_triple = df_triple[['story_triple','choices','label']]
df_triple

,story_triple,choices,label
0,"[[David, noticed, lot weight], [He, examined, ...","[After a few weeks, he started to feel much be...",0
1,"[[Tom, had, temper], [guest, made, ], [He, pun...",[Tom sat on his couch filled with regret about...,0
2,"[[Marcus, needed, clothing event], [All, were,...",[Marcus was happy to have the right clothes fo...,0
3,"[[Bobby, thought, trailer it car], [Bill, thou...",[He ended up buying the truck he wanted despit...,0
4,"[[John, was, memory], [He, tried, sermons adva...",[I drove home disappointed that my night didn'...,1
...,...,...,...
52660,"[[man, liked, flavor], [He, tried, it home], [...","[She thanked me for the recommendation., The o...",1
52661,"[[I, got, funeral trouble], [principal, said, ...","[He barely made it to his relative's., I skipp...",1
52662,"[[Janice, was, game], [She, doing, drills legs...",[The coral and the fish were all so beautiful ...,1
52663,"[[Jamie, is, ], [Jamie, wants, man], [family, ...","[Tim grinned as he saw his surprise party!, Ja...",1


In [112]:
def split (df):
    """"Duplicate all stories into 2 rows and append the correct and incorrect ending to create a binary
    classifcation task."""
    new = pd.DataFrame()
    new['story_triple'] = None
    new['choices'] = None
    new['label'] = None
    
    for index, row in tqdm(df.iterrows(),desc = 'Splitting Rows',total= len(df)):
        if df.at[index,'label'] == 0:
            ending0 = df.iloc[index].copy()
            ending0['choices'] = ending0['choices'][0]
            ending0['label'] = 1
            new.loc[len(new.index)] = ending0

            ending1 = df.iloc[index].copy()
            ending1['choices'] = ending1['choices'][1]
            ending1['label'] = 0
            new.loc[len(new.index)] = ending1

        else:
            ending0 = df.iloc[index].copy()
            ending0['choices'] = ending0['choices'][1]
            ending0['label'] = 1
            new.loc[len(new.index)] = ending0
            
            ending1 = df.iloc[index].copy()
            ending1['choices'] = ending1['choices'][0]
            ending1['label'] = 0
            new.loc[len(new.index)] = ending1
    return new


In [113]:
#Check to see if function worked
binary_df = split(df_triple)
binary_df

Splitting Rows: 100%|██████████| 52665/52665 [06:33<00:00, 133.69it/s]


,story_triple,choices,label
0,"[[David, noticed, lot weight], [He, examined, ...","After a few weeks, he started to feel much bet...",1
1,"[[David, noticed, lot weight], [He, examined, ...","In the end, he passed with flying colours.",0
2,"[[Tom, had, temper], [guest, made, ], [He, pun...",Tom sat on his couch filled with regret about ...,1
3,"[[Tom, had, temper], [guest, made, ], [He, pun...",Frank is glad he exercises more now.,0
4,"[[Marcus, needed, clothing event], [All, were,...",Marcus was happy to have the right clothes for...,1
...,...,...,...
105325,"[[Janice, was, game], [She, doing, drills legs...",The coral and the fish were all so beautiful u...,0
105326,"[[Jamie, is, ], [Jamie, wants, man], [family, ...",Jamie gets married and they spent the rest of ...,1
105327,"[[Jamie, is, ], [Jamie, wants, man], [family, ...",Tim grinned as he saw his surprise party!,0
105328,"[[orange, fell, tree], [It, hit, girl head], [...",That orange broke her nose.,1


Shuffle the DataFrame and Join Story Triples

In [114]:
# Shuffle the dataset to prevent stories from being next to each other
binary_shuffle_df = binary_df.sample(frac=1)

def join_words(triple):
    """This function will grab the triples and join them into a sentence."""

    word = [word for word in triple]
    sent1 = ' '.join(triple[0])
    sent2 = ' '.join(triple[1])
    sent3 = ' '.join(triple[2])
    sent4 = ' '.join(triple[3])
    story = sent1 + '. ' + sent2 + '. ' + sent3 + '. ' + sent4 +'.'
    story = story.lower()
    if story[0] == ' ':
        story = story[1:]
    return story

# Applying the function to each row in the DataFrame
binary_shuffle_df['story_triple'] = binary_shuffle_df['story_triple'].apply(join_words)

# Split data into training, validation, and test
binary_train_df = binary_shuffle_df[:70000]
binary_validation_df = binary_shuffle_df[70000:85000]
binary_test_df = binary_shuffle_df[85000:]

Combine Triples, Then Tokenize with Ending

In [147]:
# BERT Tokenization of training, validation, and test data

MAX_SEQUENCE_LENGTH = 50  # set max_length of the input sequence

all_train_examples = [x for x in  binary_train_df["story_triple"].to_numpy()]
all_train_endings = [x for x in  binary_train_df["choices"].to_numpy()]

x_train = tokenizer(all_train_examples, all_train_endings,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
y_train = tf.convert_to_tensor([x for x in  binary_train_df["label"].to_numpy()])


all_validation_examples = [x for x in  binary_validation_df["story_triple"].to_numpy()]
all_validation_endings = [x for x in  binary_validation_df["choices"].to_numpy()]

x_valid = tokenizer(all_validation_examples, all_validation_endings,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
y_valid = tf.convert_to_tensor([x for x in  binary_validation_df["label"].to_numpy()])


all_test_examples = [x for x in  binary_test_df["story_triple"].to_numpy()]
all_test_endings = [x for x in  binary_test_df["choices"].to_numpy()]

x_test = tokenizer(all_test_examples, all_test_endings,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
y_test = tf.convert_to_tensor([x for x in  binary_test_df["label"].to_numpy()])

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [145]:
tokenizer.decode(x_train['input_ids'][0])

'[CLS] fran riding bus herself. woman complained bags. fran became. she fighting fran. [SEP] he went to bed early. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [176]:
def create_bert_classification_model(bert_seq_model,
                                     num_train_layers=0,
                                     hidden_size = 200,
                                     dropout=0.3,
                                     learning_rate=0.00005):
    """
    Simple classification model using Bert.
    """
    if num_train_layers == 0:
        # Freeze all layers of pre-trained BERT model
        bert_seq_model.trainable = False

    elif num_train_layers == 12:
        # Train all layers of the BERT model
        bert_seq_model.trainable = True

    else:
        # Restrict training to the num_train_layers outer transformer layers
        retrain_layers = []

        for retrain_layer_number in range(num_train_layers):

            layer_code = '_' + str(11 - retrain_layer_number)
            retrain_layers.append(layer_code)


        print('retrain layers: ', retrain_layers)

        for w in bert_seq_model.weights:
            if not any([x in w.name for x in retrain_layers]):
                #print('freezing: ', w)
                w._trainable = False

    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_seq_model(bert_inputs)

    pooler_token = bert_out[0]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)


    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                 metrics=[ "acc", tf.keras.metrics.Recall()])

    return classification_model

In [177]:
bert_classification_model = create_bert_classification_model(bert_seq_model, num_train_layers=4)

bert_classification_model.summary()

retrain layers:  ['_11', '_10', '_9', '_8']
Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask_layer (Inpu  [(None, 50)]                 0         []                            
 tLayer)                                                                                          
                                                                                                  
 input_ids_layer (InputLaye  [(None, 50)]                 0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 50)]                 0         []                            
 tLayer)                                        

In [178]:
bert_classification_model_history = bert_classification_model.fit(
    [x_train.input_ids, x_train.token_type_ids, x_train.attention_mask],
    y_train,
    validation_data=([x_valid.input_ids, x_valid.token_type_ids, x_valid.attention_mask], y_valid),
    batch_size=32,
    epochs=2
)

Epoch 1/2
2188/2188 [==============================] - 3651s 2s/step - loss: 0.2267 - acc: 0.9301 - recall_4: 0.9333 - val_loss: 0.1542 - val_acc: 0.9418 - val_recall_4: 0.9208
Epoch 2/2
2188/2188 [==============================] - 3623s 2s/step - loss: 0.1255 - acc: 0.9544 - recall_4: 0.9556 - val_loss: 0.1501 - val_acc: 0.9427 - val_recall_4: 0.9378


In [179]:
bert_classification_model.evaluate([x_test.input_ids,x_test.token_type_ids,x_test.attention_mask],y_test)

636/636 [==============================] - 488s 768ms/step - loss: 0.1451 - acc: 0.9451 - recall_4: 0.9421


[0.14514827728271484, 0.9450565576553345, 0.9421303272247314]

GPT Labeled Endings

Read in GPT Data

In [265]:
import ast

def convert_to_list(text):
    return ast.literal_eval(text)

gpt_data = pd.read_csv("concat_2017_labeled.csv", converters={"cands": convert_to_list})
gpt_data.rename(columns = {'cands':'choices'}, inplace = True) 



In [ ]:
#Weird data formatting needs to be fixed before running the next function.
gpt_data.at[33934,'cands'] = ["The hitchhiker pulled out a gun and stole Jerry's car.", "The man looks at Jerry seriously saying I'm not buying it." ]
gpt_data.at[33934,'answer'] = "The man looks at Jerry seriously saying I'm not buying it."

gpt_data.at[33991,'cands'] = ["Leslie forgave James and they got engaged the next day.", "Catching him kissing another, Leslie told James it's over." ]
gpt_data.at[33991,'answer'] = "Catching him kissing another, Leslie told James it's over."

gpt_data.at[34651,'cands'] = ["Wow, thank you for the guitar, Marcy, that's just what I wanted!", "Is that a ukulele? I hate ukuleles!"]
gpt_data.at[34651,'answer'] = "Wow, thank you for the guitar, Marcy, that's just what I wanted!"

gpt_data.at[36972,'cands'] = ["They realized their daughter was actually an alien and had been trying to communicate with them.", "They understood she was saying, Where's the baby?"]
gpt_data.at[36972,'answer'] = "They understood she was saying, Where's the baby?"

gpt_data.at[40320,'cands'] = ["Jonathan's dad said, You will not be driving for a very long time.", "Sara and Jonathan fell in love at first sight."]
gpt_data.at[40320,'answer'] = "Jonathan's dad said, You will not be driving for a very long time."

In [267]:
# Create label to identify the index of the correct answer in the "cands" column
def get_label(df):
    df["label"] = None
    for index,row in tqdm(df.iterrows(),desc = 'label',total = len(df)):
        df.at[index,"label"] = df.at[index,"choices"].index(df.at[index,"answer"])
    return df

gpt_prepped_df = get_label(gpt_data)

label: 100%|██████████| 52664/52664 [00:06<00:00, 8239.72it/s]


Create Triples for GPT

In [299]:
# Fix strange formatting in story sentences before graphing.
gpt_prepped_df.at[43888,'sentence4'] = 'Although dejected, Jared kept his head high.'
gpt_prepped_df.at[44230,'sentence4'] = 'When he placed one sticker, it stuck to his hand.'
gpt_prepped_df.at[46883,'sentence3'] = "When the name was called for the award though, it wasn't his."

gpt_story_trpl_df = df_graph(gpt_prepped_df)

gpt_story_trpl_df.to_json("gpt_stories_triple.json")
gpt_story_trpl_df = pd.read_json('ROCStories_winter2017_mixed_tripled.json')

In [308]:
def all_together(df):
  '''This function will package together all the functions created to produce a final result.
  The df must have columns: sentence1,	sentence2,	sentence3,	sentence4, choices, and label.
  Not included in here is the creation of story (joint of sent1-sent4) or choices[list of both endings]'''
  df = tokenize_for_cosine(df)
  print("\nDone w/ Token\n")
  df = cosine_scoring(df)
  print("\nDone w/ Scoring\n")
  final = get_accuracy(df)
  return final

all_together(gpt_story_trpl_df)

Tokenizing:   0%|          | 0/52664 [00:00<?, ?it/s]

Tokenizing: 100%|██████████| 52664/52664 [2:41:54<00:00,  5.42it/s]  



Done w/ Token



Scoring: 100%|██████████| 52664/52664 [00:30<00:00, 1721.03it/s]



Done w/ Scoring



(35181, 35181, 17483, 17483, 0.6680274950630412, 0.6680274950630412)

Seq2Seq Classification w/ GPT Endings

In [312]:
from transformers import TFBertForSequenceClassification

id2label = {0: "Invalid", 1: "Valid"}
label2id = {val: key for key, val in id2label.items()}

bert_seq_model = TFBertForSequenceClassification.from_pretrained(
    './bert_model_weights', num_labels=2, id2label=id2label, label2id=label2id)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [309]:
gpt_seq_df = pd.read_json('gpt_stories_triple.json')
gpt_seq_df

,Unnamed: 0,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,answer,choices,story,label,story_triple
0,0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...",[His weight continued to increase and he devel...,David noticed he had put on a lot of weight re...,1,"[[David, noticed, lot weight], [He, examined, ..."
1,1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...,[Tom sat on his couch filled with regret about...,Tom had a very short temper. One day a guest m...,0,"[[Tom, had, temper], [guest, made, ], [He, pun..."
2,2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...,[Marcus was happy to have the right clothes fo...,Marcus needed clothing for a business casual e...,0,"[[Marcus, needed, clothing event], [All, were,..."
3,3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...,"[He listened to Bobby and bought a trailer, re...",Bobby thought Bill should buy a trailer and ha...,1,"[[Bobby, thought, trailer it car], [Bill, thou..."
4,4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.,[His congregation ridiculed him and he was ask...,John was a pastor with a very bad memory. He t...,1,"[[John, was, memory], [He, tried, sermons adva..."
...,...,...,...,...,...,...,...,...,...,...,...,...
46495,46495,e9db07c9-3ea2-438f-853c-318624595703,Snapchats,Yenny took tons of snapchats that day.,Her friend messaged her saying she was being a...,Yenny said she was just proud of her face.,Yenny noticed many people were unfollowing her.,She realized she needed to lay off the snapchats.,[ Yenny became a social media sensation overni...,Yenny took tons of snapchats that day. Her fri...,1,"[[Yenny, took, tons snapchats], [friend, messa..."
46496,46496,ab8fe175-9e56-4d89-a3b6-6ba3b7fd3f45,The Baseball,Javier loved playing outside.,His dad bought him a baseball and told him to ...,He made up lots of fun games to play with it.,He would throw it high into the air and then c...,Javier's baseball was his favorite new toy.,"[Javier's baseball was his favorite new toy., ...",Javier loved playing outside. His dad bought h...,0,"[[Javier, loved, ], [dad, bought, baseball him..."
46497,46497,0f5b5ec2-6e95-4ddd-b252-8d1bdcea02d8,Argument,Danny and his girlfriend got into an argument.,Danny thought she was cheating on him.,Danny dumped her.,He realized later that he was being stupid.,He took her back.,[He found out she was actually cheating on him...,Danny and his girlfriend got into an argument....,1,"[[Danny, got, argument], [Danny, thought, him]..."
46498,46498,d5d2bec0-b8b7-4a1f-bd58-48bba22f48a5,Bull Run,"Todd, Dick, and Tim were walking to the river ...",They decided to take a shortcut across a pasture.,They did not know a farmer's bull was living t...,The angry bull chased them all of the way acro...,They made it through the fence just in time.,[The bull caught up to them and

In [314]:
#Create two rows per story, each with a different story ending
binary_gpt_df = split(gpt_seq_df) 

# Randomize the df, so the're not in order
binary_gpt_shuffle_df = binary_gpt_df.sample(frac=1)

# Apply function to join the triples into a sentence.
binary_gpt_shuffle_df['story_triple'] = binary_gpt_shuffle_df['story_triple'].apply(join_words)

#Split the dataframe into training, validation, and test
binary_gpt_train_df = binary_gpt_shuffle_df[:70000]
binary_gpt_validation_df = binary_gpt_shuffle_df[70000:85000]
binary_gpt_test_df = binary_gpt_shuffle_df[85000:]

In [ ]:
# BERT Tokenization of training, validation, and test gpt data

MAX_SEQUENCE_LENGTH = 50  # set max_length of the input sequence

all_train_examples = [x for x in  binary_gpt_train_df["story_triple"].to_numpy()]
all_train_endings = [x for x in  binary_gpt_train_df["choices"].to_numpy()]

x_train = tokenizer(all_train_examples, all_train_endings,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
y_train = tf.convert_to_tensor([x for x in  binary_gpt_train_df["label"].to_numpy()])


all_validation_examples = [x for x in  binary_gpt_validation_df["story_triple"].to_numpy()]
all_validation_endings = [x for x in  binary_gpt_validation_df["choices"].to_numpy()]

x_valid = tokenizer(all_validation_examples, all_validation_endings,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
y_valid = tf.convert_to_tensor([x for x in  binary_gpt_validation_df["label"].to_numpy()])


all_test_examples = [x for x in  binary_gpt_test_df["story_triple"].to_numpy()]
all_test_endings = [x for x in  binary_gpt_test_df["choices"].to_numpy()]

x_test = tokenizer(all_test_examples, all_test_endings,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
y_test = tf.convert_to_tensor([x for x in  binary_gpt_test_df["label"].to_numpy()])

In [319]:
bert_classification_model = create_bert_classification_model(bert_seq_model, num_train_layers=12)

bert_classification_model_history = bert_classification_model.fit(
    [x_train.input_ids, x_train.token_type_ids, x_train.attention_mask],
    y_train,
    validation_data=([x_valid.input_ids, x_valid.token_type_ids, x_valid.attention_mask], y_valid),
    batch_size=32,
    epochs=2
)

Epoch 1/2
2188/2188 [==============================] - 6829s 3s/step - loss: 0.5138 - acc: 0.7746 - recall_5: 0.7733 - val_loss: 0.4852 - val_acc: 0.8003 - val_recall_5: 0.7977
Epoch 2/2
2188/2188 [==============================] - 6785s 3s/step - loss: 0.4442 - acc: 0.8252 - recall_5: 0.8217 - val_loss: 0.4802 - val_acc: 0.7984 - val_recall_5: 0.8146


In [320]:
bert_classification_model.evaluate([x_test.input_ids,x_test.token_type_ids,x_test.attention_mask],y_test)

636/636 [==============================] - 505s 794ms/step - loss: 0.4749 - acc: 0.8033 - recall_5: 0.8182


[0.4748801290988922, 0.8033254742622375, 0.8181727528572083]